In [1]:
###
### PYTHON PACKAGES
###
import pandas as pd
import re
import emoji
from ftfy import fix_text 
import unidecode
from langdetect import detect, detect_langs
import spacy
from spacy import displacy

###
### USER-DEVELOPED IMPORTS
###
# from project import *
from capture_group import *
from output_format import *

In [26]:
cleaned_data = pd.read_csv('text_cleaned.csv')['text']
win_data = pd.read_csv('wins.csv')['text']
spacy_data = pd.read_csv('spacy_info.csv')

In [45]:
from itertools import islice

account_rts = {}

for text in cleaned_data:
    tweet = text.split(' ')
    if tweet[0] == 'RT':
        if tweet[1] in account_rts:
            account_rts[tweet[1]] += 1
        else:
            account_rts.update({tweet[1]:1})

account_dicts = dict(sorted(account_rts.items(), key=lambda item: item[1], reverse=True))

top_accounts = list(islice(account_dicts, 20))
top_accounts

print(top_accounts)

['@goldenglobes:', '@eonline:', '@PerezHilton:', '@TheEllenShow:', '@EmWatson:', '@nbcsnl:', '@CNNshowbiz:', '@VanityFair:', '@EW:', '@BuzzFeed:', '@vulture:', '@nbc:', '@piersmorgan:', '@MARLONLWAYANS:', '@HuffingtonPost:', '@buckhollywood:', '@MarilynMonroeES:', '@DavidSpade:', '@THR:', '@MTVNews:']


In [46]:
text = "RT @goldenglobes: Best Actor in a Motion Picture - Comedy or Musical - Hugh Jackman (@RealHughJackman) - Les Miserables - #GoldenGlobes"

In [47]:
retweet_pattern = r'RT\s' + '|'.join(top_accounts)
reputable_df = cleaned_data.apply(lambda text: text if re.search(retweet_pattern, text, re.IGNORECASE) else None)

In [48]:
reputable_df.dropna(inplace=True)

In [49]:
keys = r"(\bwin\b|\bwins\b|\bwon\b|\bawarded\b|\baward\b|\bprize\b|\bhonor\b|\bmedal\b|\btrophy\b|\bbest)"
reputable_df = reputable_df.apply(lambda x: x if re.search(keys, x.lower()) != None else None)
reputable_df.dropna(inplace=True)

In [50]:
reputable_df.to_csv("reputable_sources_only.csv")

In [51]:
text = 'RT @goldenglobes: Best Actor in a Motion Picture - Comedy or Musical - Hugh Jackman (@RealHughJackman) - Les Miserables - #GoldenGlobes'

In [52]:
if re.search(keys, text.lower()) != None:
    print(text)
else:
    print("fail")

RT @goldenglobes: Best Actor in a Motion Picture - Comedy or Musical - Hugh Jackman (@RealHughJackman) - Les Miserables - #GoldenGlobes


In [53]:
results = reputable_df.apply(find_award_winner)

In [54]:
results.dropna(inplace=True)

In [55]:
results.apply(extract_answers)

12350     None
12370     None
12373     None
12459     None
12610     None
          ... 
152151    None
152600    None
152741    None
152749    None
152850    None
Name: text, Length: 1090, dtype: object

In [57]:
top_awards = dict(sorted(awards_list.items(), key=lambda item: item[1].votes, reverse=True))
for i in top_awards:
    print(top_awards[i].output())

{'best actor in a motion picture': {'nominees': [], 'presenters': [], 'winner': [('hugh jackman', 128), ('daniel day lewis', 18)], 'votes': 146}}
{'best song at the': {'nominees': [], 'presenters': [], 'winner': [('adele', 144)], 'votes': 144}}
{'best supporting actress in a motion picture is': {'nominees': [], 'presenters': [], 'winner': [('anne hathaway', 114), ('sudah ku', 2), ('yayyyy', 2)], 'votes': 118}}
{'best director motion picture': {'nominees': [], 'presenters': [], 'winner': [('goldenglobe', 102)], 'votes': 102}}
{'best director': {'nominees': [], 'presenters': [], 'winner': [('affleck', 2), ('ben affleck', 74)], 'votes': 76}}
{'best motion picture': {'nominees': [], 'presenters': [], 'winner': [('les miserables', 39), ('lesmis', 4), ('goldenglobes', 10), ('argo', 12), ('the film', 4)], 'votes': 69}}
{'best actor': {'nominees': [], 'presenters': [], 'winner': [('damien lewis', 42), ('hugh jackman', 2), ('daniel day lewis', 22), ('film wajib tonton ini', 1)], 'votes': 67}}
{